In [1]:
!pip install flask

In [2]:
import datetime
import hashlib
import json
import flask
from flask import Flask, jsonify
import random

In [5]:
r = 'quickbrownfox'
r

'quickbrownfox'

In [6]:
r[:5]

'quick'

In [88]:
new_proof = 1
previous_proof = 1
str(new_proof**2 - previous_proof**2).encode()

b'0'

In [50]:
class Blockchain:
    def __init__(self):
        self.chain = []
        self.create_block(proof = 1, previous_hash = '0')
        
    def create_block(self, proof, previous_hash):
        block = {'index': len(self.chain) + 1,
                 'timestamp': str(datetime.datetime.now()),
                 'proof': proof,
                 'previous_hash': previous_hash
                }
        self.chain.append(block)
        return block
    
    def print_previous_block(self):
        return self.chain[-1]
    
    def proof_of_work(self, previous_proof):
        new_proof = 1
        check_proof = False
        
        while check_proof is False:
            hash_operation = hashlib.sha256(
                str(new_proof**2 - previous_proof**2).encode()).hexdigest()
            #print(hash_operation[:10])
            if hash_operation[:5] == '5fece':
                check_proof = True
            else:
                new_proof += 1
        return new_proof
    
    def hash(self, block):
        encoded_block = json.dumps(block, sort_keys = True).encode()
        return hashlib.sha256(encoded_block).hexdigest()
    
    def chain_valid(self, chain):
        previous_block = chain[0]
        block_index = 1
        block = chain[block_index]
        
        print("Previous hash: ", block['previous_hash'])
        print("this object's previous block hash: ", self.hash(previous_block))

        while block_index < len(chain):
            block = chain[block_index]
            print("block: ", block)
            if block['previous_hash'] != self.hash(previous_block):
                print("Hashes do not match.")
                return False
            
            previous_proof = previous_block['proof']
            proof = block['proof']
            hash_operation = hashlib.sha256(
                str(proof**2 - previous_proof**2).encode()).hexdigest()
        
            print(hash_operation)
            if hash_operation[:5] != '5fece':
                print("Proofs don't match. Hash does not correspond to zero.")
                return False
        
            previous_block = block
            block_index += 1
        
        return True
        

In [51]:
bc = Blockchain()

In [67]:
previous_block = bc.print_previous_block()
previous_block

{'index': 2,
 'timestamp': '2023-01-26 20:25:22.194263',
 'proof': 1,
 'previous_hash': '3b4f5c3f5a2a4b6200f07ef932002a557058204a0aa08e8c37a7a15bcc6f716b'}

In [53]:
previous_proof = previous_block['proof']
previous_proof

1

In [54]:
proof = bc.proof_of_work(previous_proof)
proof

1

In [55]:
previous_hash = bc.hash(previous_block)
previous_hash

'3b4f5c3f5a2a4b6200f07ef932002a557058204a0aa08e8c37a7a15bcc6f716b'

In [56]:
block = bc.create_block(proof, previous_hash)
block

{'index': 2,
 'timestamp': '2023-01-26 20:25:22.194263',
 'proof': 1,
 'previous_hash': '3b4f5c3f5a2a4b6200f07ef932002a557058204a0aa08e8c37a7a15bcc6f716b'}

In [57]:
bc.chain_valid(bc.chain)

Previous hash:  3b4f5c3f5a2a4b6200f07ef932002a557058204a0aa08e8c37a7a15bcc6f716b
this object's previous block hash:  3b4f5c3f5a2a4b6200f07ef932002a557058204a0aa08e8c37a7a15bcc6f716b
block:  {'index': 2, 'timestamp': '2023-01-26 20:25:22.194263', 'proof': 1, 'previous_hash': '3b4f5c3f5a2a4b6200f07ef932002a557058204a0aa08e8c37a7a15bcc6f716b'}
5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9


True

In [58]:
prev_proof = bc.chain[0]['proof']
prev_proof

1

In [59]:
now_proof = bc.chain[1]['proof']
now_proof

1

In [ ]:
app = Flask(__name__)
bc2 = Blockchain()

In [ ]:
@app.route('/mine_block', methods=['GET'])
def mine_block():
    previous_block = bc2.print_previous_block()
    previous_proof = previous_block['proof']
    proof = bc2.proof_of_work(previous_proof)
    previous_hash = bc2.hash(previous_block)
    block = bc2.create_block(proof, previous_hash)
    
    response = {'message': 'A block is MINED',
                'index': block['index'],
                'timestamp': block['timestamp'],
                'proof': block['proof'],
                'previous_hash': block['previous_hash']}
    
    return jsonify(response), 200

@app.route('/get_chain', methods=['GET'])
def display_chain():
    response = {'chain': bc2.chain,'length': len(bc2.chain)}
    return jsonify(response), 200

# Check validity of blockchain 
@app.route('/valid', methods=['GET'])
def valid():
    valid = bc2.chain_valid(bc2.chain)
    if valid:
        response = {'message': 'The Blockchain is valid.'}
    else:
        response = {'message': 'The Blockchain is not valid.'}
    return jsonify(response), 200

# Run the flask server locally
app.run(host='127.0.0.1', port=5000)